In [1]:
#!pip install kafka-python

In [2]:
#!pip install s3fs

In [3]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps, loads
import json
from itertools import cycle
import os

In [4]:
producer = KafkaProducer(bootstrap_servers = ['52.32.219.245:9092'], 
                          value_serializer = lambda x: dumps(x).encode('utf-8'))

In [5]:
#producer.send('demo_kaf1', value = {'surname': 'angelina'})

In [6]:
#producer.flush()

In [7]:
path = os.getcwd()
PARENT_DIR = os.path.abspath(os.path.join(path, os.pardir))
PARENT_DIR

'/Users/hasana/Documents/projects/Data-Engineering-Projects/aws/kafka'

In [8]:


# Set the directory where the CSV files are stored
directory = PARENT_DIR + "/data repo"

# Create an empty list to store the data frames
dfs = []

# Loop through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a data frame
        df = pd.read_csv(os.path.join(directory, filename))
        # Append the data frame to the list
        dfs.append(df)

# Concatenate all data frames in the list into a single data frame
master_df = pd.concat(dfs, ignore_index=True)

drop_list = ['Product', 'LG Washing Machine', 'LG Dryer', 'Flatscreen TV', 'Vareebadd Phone']
mask = master_df.isin(drop_list).any(axis=1)

# Drop the rows based on the boolean mask
master_df = master_df[~mask]
master_df = master_df.dropna()

In [9]:
# create dictionary mapping original values to unique values
category_product = zip(list(master_df.Product.unique()),
                       ['Laptop', 'Charger', 'Monitor', 
                      'Battery', 'Headphone', 'Battery', 'Laptop','Charger',
                     'Phone', 'Headphone','Headphone','Phone',
                      'Monitor','Monitor','Monitor' ])

mapping_dict = dict(category_product)

# use map method to create new column
master_df['Category'] = master_df['Product'].map(mapping_dict)

In [10]:
while True:
    #sample_records = master_df.sample(1).to_dict(orient="records")
    producer.send('demo_kaf1', master_df.sample(1).to_dict(orient="records"))
    sleep(5)

KeyboardInterrupt: 

In [11]:
producer.flush()